In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ukwud\\OneDrive - Aston University\\Documents\\GitHub\\text-summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ukwud\\OneDrive - Aston University\\Documents\\GitHub\\text-summarizer'

In [5]:
#entity is the return type of the function

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])
            
    def get_data_ingestion_config(self) -> DataIngestionConfig:
            config = self.config.data_ingestion

            create_directories([config.root_dir])

            data_ingestion_config = DataIngestionConfig(
                  root_dir=config.root_dir,
                  source_URL=config.source_URL,
                  local_data_file=config.local_data_file,
                  unzip_dir=config.unzip_dir
      )

            return data_ingestion_config

update components

In [12]:
import os
import urllib.request as request
import zipfile
from text_summarizer.logging import logger
from text_summarizer.utils.common import get_size

In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

            logger.info(f"{filename} has been downloaded with the following info: \n{headers}")

        else:
            logger.info(f"file already existed of file size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)


Create the pipeline

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-06 22:43:44,942: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-06 22:43:44,944: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-06 22:43:44,946: INFO: common: directory created at: artifacts]
[2025-01-06 22:43:44,947: INFO: common: directory created at: artifacts/data_ingestion]
[2025-01-06 22:43:46,418: INFO: 1057346680: artifacts/data_ingestion/data.zip has been downloaded with the following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: AD88:08E4:F41599:1429A27:677C5CA1
Accept-Ranges: bytes
Date: Mon, 06 Jan 2025 22:43:46 GMT
Via: 1.1 varnish
X-Served-By: cache-lon420111-